In [8]:
import sys
import time,os

sys.path.append("..")  # make sure we can run this from the repo
from IPython.display import Markdown, clear_output, display
from ipywidgets import widgets

import cohere
cohere.COHERE_API_URL = 'https://stg.api.cohere.ai'

model = 'chat-6b-v10-2-specialtokens'

CHAT_MODEL = 'converse-xlarge-nightly'

## Client example

### Simple Generate Example

In [9]:
co = cohere.Client(os.environ['CO_STG_API_KEY'])
start_time = time.time()
streaming_gens = co.generate(prompt="Hey! Don't worry, 🐝 happy~", max_tokens=200, model=model,stream=True)
for i, token in enumerate(streaming_gens):
    print(f"[{i}][{time.time()-start_time:.2f}s] {token}")
# the request id is available after the first token has streamed, and response so far is in texts
streaming_gens.ids, streaming_gens.texts

[0][0.53s] StreamingText(id='48669cbc-1e84-4e8b-aebf-5c97a6a75cda', index=0, text='\n')
[1][0.54s] StreamingText(id=None, index=0, text='It')
[2][0.55s] StreamingText(id=None, index=0, text="'s")
[3][0.56s] StreamingText(id=None, index=0, text=' a')
[4][0.57s] StreamingText(id=None, index=0, text=' cute')
[5][0.58s] StreamingText(id=None, index=0, text=' little')
[6][0.59s] StreamingText(id=None, index=0, text=' alien')
[7][0.61s] StreamingText(id=None, index=0, text='!')
[8][0.62s] StreamingText(id=None, index=0, text=' It')
[9][0.63s] StreamingText(id=None, index=0, text=' was')
[10][0.64s] StreamingText(id=None, index=0, text=' too')
[11][0.65s] StreamingText(id=None, index=0, text=' scared')
[12][0.66s] StreamingText(id=None, index=0, text=' to')
[13][0.68s] StreamingText(id=None, index=0, text=' come')
[14][0.69s] StreamingText(id=None, index=0, text=' out')
[15][0.70s] StreamingText(id=None, index=0, text=',')
[16][0.71s] StreamingText(id=None, index=0, text=' but')
[17][0.74s] S

(['48669cbc-1e84-4e8b-aebf-5c97a6a75cda'],
 ["\nIt's a cute little alien! It was too scared to come out, but it sure is cute! I hope you can take pictures of it for me!"])

### Widgets Chat Example

In [ ]:
co = cohere.Client()
max_tokens = 250

text = ""
output_area = display("Type your message below and hit enter to chat!", display_id=True)
input_text = widgets.Text(value="Hey!", description="User:", disabled=False)

def append_text(s):
    global text
    text += s
    output_area.update(Markdown(text))

def stream_response(input_text):
    query = input_text.value
    if query.startswith("<generating"):
        return
    input_text.value = "<generating response>"
    append_text(f"\n\nUser: {query}\n\nChatbot: ")
    streaming_chat = co.chat(
        query=query,
        max_tokens=max_tokens,
        model=CHAT_MODEL,
        stream=True,
    )
    for token in streaming_chat:
        append_text(token.text)
    input_text.value = ""

display(input_text)
input_text.on_submit(stream_response)

## AsyncClient example

### Widgets generate example

In [13]:
prompt = "안녕하"
num_generations = 5
max_tokens = 50
texts = [f"**Generation #{i+1}:** {prompt}" for i in range(num_generations)]
displays = [display(display_id=True) for t in texts]

async with cohere.AsyncClient(os.environ['CO_STG_API_KEY']) as aio_co:
    start_time = time.time()
    aio_streaming_gens = await aio_co.generate(
        prompt=prompt,
        max_tokens=max_tokens,
        #model="command-xlarge-nightly",
        num_generations=num_generations,
        model=model,
        stream=True,
    )

    async for token in aio_streaming_gens:
        texts[token.index] += token.text
        displays[token.index].update(Markdown(texts[token.index]))

**Generation #1:** 안녕하 좋은 상황 준비 준비 회화 중화민주주주의회

**Generation #2:** 안녕하 주중대민회비 중외 중외 중중 파테프루 중

**Generation #3:** 안녕하 Hello, my name is Rachel and I am the Student Director for the U-M School of Social Work's Student Leadership Council. I am currently a senior in the MSW program at the U-M School of Social Work, and I will graduate

**Generation #4:** 안녕하 미주철료 차대 미주철료 차대 미주철료 차

**Generation #5:** 안녕하 유머체중 왕왕 중국 후화왕대통화 철통된

None


In [12]:
aio_streaming_gens

cohere.StreamingGenerations {
	response: <ClientResponse(https://stg.api.cohere.ai/v1/generate) [401 Unauthorized]>
<CIMultiDictProxy('Content-Type': 'application/json', 'Vary': 'Origin', 'x-ratelimit-limit': '10000000', 'x-ratelimit-remaining': '9999994', 'x-ratelimit-reset': '1674403260', 'Date': 'Mon, 27 Mar 2023 21:14:40 GMT', 'Content-Length': '31', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000')>

	ids: [None]
	texts: ['']
}

### Simple Chat Example

In [ ]:
query = "Do You Want to Build a Snowman?"
async with cohere.AsyncClient() as aio_co:
    streaming_chat = await aio_co.chat(
        query=query,
        max_tokens=50,
        model=CHAT_MODEL,
        stream=True,
    )
    async for token in streaming_chat:
        print(token)
    print(streaming_chat.texts)